# **💁🏻🗨️💁🏻‍♂️대화 요약 Baseline code**
> **Dialogue Summarization** 경진대회에 오신 여러분 환영합니다! 🎉    
> 본 대회에서는 최소 2명에서 최대 7명이 등장하여 나누는 대화를 요약하는 BART 기반 모델의 baseline code를 제공합니다.     
> 주어진 데이터를 활용하여 일상 대화에 대한 요약을 효과적으로 생성하는 모델을 만들어봅시다!

## ⚙️ 데이터 및 환경설정

### 1) 필요한 라이브러리 설치

- 필요한 라이브러리를 설치한 후 불러옵니다.

In [ ]:
import pandas as pd
import os
import re
import json
import yaml
from glob import glob
from tqdm import tqdm
from pprint import pprint
import torch
import pytorch_lightning as pl
from rouge import Rouge # 모델의 성능을 평가하기 위한 라이브러리입니다.

from torch.utils.data import Dataset , DataLoader
from transformers import AutoTokenizer, BartForConditionalGeneration, BartConfig
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import Trainer, TrainingArguments
from transformers import EarlyStoppingCallback

import wandb # 모델 학습 과정을 손쉽게 Tracking하고, 시각화할 수 있는 라이브러리입니다.

### 2) Config file 만들기 (선택)
- 모델 생성에 필요한 다양한 매개변수 정보를 저장할 수 있습니다.  
  따라서, 코드 상에서 모델의 매개변수를 설정할 수도 있지만 독립적인 매개변수 정보 파일을 생성하여 관리할 수 있습니다.

In [138]:

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


# T5 모델 로드
model = AutoModelForSeq2SeqLM.from_pretrained('eenzeenee/t5-base-korean-summarization')
tokenizer = AutoTokenizer.from_pretrained('eenzeenee/t5-base-korean-summarization')


In [139]:
config_data = {
    "general": {
        "data_path": "/root/data/",
        "model_name": "eenzeenee/t5-base-korean-summarization",
        "output_dir": "/root/output/"
    },
    "tokenizer": {
        "encoder_max_len": 512,
        "decoder_max_len": 100,
        "special_tokens": ['#Address#', '#CarNumber#', '#CardNumber#', '#DateOfBirth#', '#Email#', '#PassportNumber#', '#Person#', '#Person1#', '#Person2#', '#Person3#', '#Person4#', '#Person5#', '#Person6#', '#Person7#', '#PhoneNumber#', '#SSN#']
    },
    "training": {
        "overwrite_output_dir": True,
        "num_train_epochs": 5,  # Seq2SeqTrainingArguments 기반 설정
        "learning_rate": 4e-5,  # Seq2SeqTrainingArguments 기반 설정
        "per_device_train_batch_size": 8,  # Seq2SeqTrainingArguments 기반 설정
        "per_device_eval_batch_size": 8,  # Seq2SeqTrainingArguments 기반 설정
        "auto_find_batch_size": False,  # Seq2SeqTrainingArguments 기반 설정
        "warmup_ratio": 0.1,
        "weight_decay": 0.01,  # Seq2SeqTrainingArguments 기반 설정
        "lr_scheduler_type": 'linear',  # Seq2SeqTrainingArguments 기반 설정
        "optim": 'adamw_torch',
        "gradient_accumulation_steps": 1,
        "evaluation_strategy": 'epoch',
        "save_strategy": 'epoch',
        "save_total_limit": 5,
        "fp16": True,  # Seq2SeqTrainingArguments 기반 설정
        "load_best_model_at_end": True,
        "seed": 42,
        "logging_dir": "./logs",
        "logging_strategy": "epoch",
        "predict_with_generate": True,
        "generation_max_length": 100,
        "do_train": True,
        "do_eval": True,
        "early_stopping_patience": 3,
        "early_stopping_threshold": 0.001,
        "report_to": "wandb"
    },
    "wandb": {
        "entity": "dksjj123",
        "project": "NLP",
        "name": "test"
    },
    "inference": {
        "ckt_path": "/root/output/",
        "result_path": "/root/prediction/",
        "no_repeat_ngram_size": 2,
        "early_stopping": True,
        "generate_max_length": 100,
        "num_beams": 4,
        "batch_size": 32,
        "remove_tokens": [f"{tokenizer.pad_token}"]
    }
}


- 참고✅    
: wandb 라이브러리를 사용하기 위해선 entity, project, name를 지정해주어야 합니다. wandb 홈페이지에 가입한 후 얻은 정보를 입력하여 작동할 수 있습니다.

In [140]:
# 모델의 구성 정보를 YAML 파일로 저장합니다.
config_path = "root/config.yaml"
with open(config_path, "w") as file:
    yaml.dump(config_data, file, allow_unicode=True)

### 3) Configuration 불러오기

In [141]:
# 저장된 config 파일을 불러옵니다.
config_path = "/root/config.yaml"

with open(config_path, "r") as file:
    loaded_config = yaml.safe_load(file)

# 불러온 config 파일의 전체 내용을 확인합니다.
pprint(loaded_config)

{'general': {'data_path': '/root/data/',
             'model_name': 'eenzeenee/t5-base-korean-summarization',
             'output_dir': '/root/output/'},
 'inference': {'batch_size': 32,
               'ckt_path': '/root/output/',
               'early_stopping': True,
               'generate_max_length': 100,
               'no_repeat_ngram_size': 2,
               'num_beams': 4,
               'remove_tokens': ['<pad>'],
               'result_path': '/root/prediction/'},
 'tokenizer': {'decoder_max_len': 100,
               'encoder_max_len': 512,
               'special_tokens': ['#Address#',
                                  '#CarNumber#',
                                  '#CardNumber#',
                                  '#DateOfBirth#',
                                  '#Email#',
                                  '#PassportNumber#',
                                  '#Person#',
                                  '#Person1#',
                                  '#Person2#',
    

In [142]:
# 실험에 쓰일 데이터의 경로, 사용될 모델, 모델의 최종 출력 결과를 저장할 경로에 대해 확인합니다.
loaded_config['general']

{'data_path': '/root/data/',
 'model_name': 'eenzeenee/t5-base-korean-summarization',
 'output_dir': '/root/output/'}

In [143]:
# 이곳에 사용자가 저장한 데이터 dir 설정하기
loaded_config['general']['data_path'] = "/root/data"

In [144]:
# 데이터 전처리를 하기 위해 tokenization 과정에서 필요한 정보들을 확인합니다.
loaded_config['tokenizer']

{'decoder_max_len': 100,
 'encoder_max_len': 512,
 'special_tokens': ['#Address#',
  '#CarNumber#',
  '#CardNumber#',
  '#DateOfBirth#',
  '#Email#',
  '#PassportNumber#',
  '#Person#',
  '#Person1#',
  '#Person2#',
  '#Person3#',
  '#Person4#',
  '#Person5#',
  '#Person6#',
  '#Person7#',
  '#PhoneNumber#',
  '#SSN#']}

In [145]:
# 모델이 훈련 시 적용될 매개변수를 확인합니다.
loaded_config['training']

{'auto_find_batch_size': False,
 'do_eval': True,
 'do_train': True,
 'early_stopping_patience': 3,
 'early_stopping_threshold': 0.001,
 'evaluation_strategy': 'epoch',
 'fp16': True,
 'generation_max_length': 100,
 'gradient_accumulation_steps': 1,
 'learning_rate': 4e-05,
 'load_best_model_at_end': True,
 'logging_dir': './logs',
 'logging_strategy': 'epoch',
 'lr_scheduler_type': 'linear',
 'num_train_epochs': 5,
 'optim': 'adamw_torch',
 'overwrite_output_dir': True,
 'per_device_eval_batch_size': 8,
 'per_device_train_batch_size': 8,
 'predict_with_generate': True,
 'report_to': 'wandb',
 'save_strategy': 'epoch',
 'save_total_limit': 5,
 'seed': 42,
 'warmup_ratio': 0.1,
 'weight_decay': 0.01}

In [146]:
# 모델 학습 과정에 대한 정보를 제공해주는 wandb 설정 내용을 확인합니다.
loaded_config['wandb']

{'entity': 'dksjj123', 'name': 'test', 'project': 'NLP'}

In [147]:
# (선택) 이곳에 사용자가 사용할 wandb config 설정
loaded_config['wandb']['entity'] = "dksjj123"
loaded_config['wandb']['name'] = "NLP_real"
loaded_config['wandb']['project'] = "test"

In [148]:
# 모델이 최종 결과를 출력하기 위한 매개변수 정보를 확인합니다.
loaded_config['inference']

{'batch_size': 32,
 'ckt_path': '/root/output/',
 'early_stopping': True,
 'generate_max_length': 100,
 'no_repeat_ngram_size': 2,
 'num_beams': 4,
 'remove_tokens': ['<pad>'],
 'result_path': '/root/prediction/'}

### 4) 데이터 불러와서 확인해보기
- 실험에서 쓰일 데이터를 load하여 데이터의 구조와 내용을 살펴보겠습니다.
- Train, dev, test 순서대로 12457, 499, 250개 씩 데이터가 구성되어 있습니다.

In [149]:
# config에 저장된 데이터 경로를 통해 train과 validation data를 불러옵니다.
data_path = loaded_config['general']['data_path']

# train data의 구조와 내용을 확인합니다.
train_df = pd.read_csv(os.path.join(data_path,'train.csv'))
train_df.tail()

,fname,dialogue,summary,topic
12452,train_12455,#Person1#: 실례합니다. 맨체스터 출신의 그린 씨이신가요?\n#Person2...,탄 링은 흰머리와 수염으로 쉽게 인식되는 그린 씨를 만나 호텔로 데려갈 예정입니다....,누군가를 태우다
12453,train_12456,#Person1#: 이윙 씨가 우리가 컨퍼런스 센터에 오후 4시에 도착해야 한다고 ...,#Person1#과 #Person2#는 이윙 씨가 늦지 않도록 요청했기 때문에 컨퍼...,컨퍼런스 센터
12454,train_12457,#Person1#: 오늘 어떻게 도와드릴까요?\n#Person2#: 차를 빌리고 싶...,#Person2#는 #Person1#의 도움으로 5일 동안 소형 차를 빌립니다.,차 렌트
12455,train_12458,#Person1#: 오늘 좀 행복해 보이지 않아. 무슨 일 있어?\n#Person2...,#Person2#의 엄마가 일자리를 잃었다. #Person2#는 엄마가 우울해하지 ...,실직
12456,train_12459,"#Person1#: 엄마, 다음 토요일에 이 삼촌네 가족을 방문하기 위해 비행기를 ...",#Person1#은 다음 토요일에 이 삼촌네를 방문할 때 가방을 어떻게 싸야 할지 ...,짐 싸기


In [150]:
# validation data의 구조와 내용을 확인합니다.
val_df = pd.read_csv(os.path.join(data_path,'dev.csv'))
val_df.tail()

,fname,dialogue,summary,topic
494,dev_495,#Person1#: 이제 새해가 되어서 새로운 시작을 하려고 결심했어. \r\n#P...,#Person1#은 새해에 금연을 하고 커밍아웃하기로 결정했습니다. #Person2...,새해
495,dev_496,"#Person1#: 너, 조랑 결혼했지? \r\n#Person2#: 조? 무슨 말인...",#Person1#은 #Person2#가 조와 결혼했다고 생각했다. #Person2#...,사랑에 빠지다
496,dev_497,"#Person1#: 무엇을 도와드릴까요, 부인?\r\n#Person2#: 몇 주 동...",#Person2#의 차에서 이상한 소리가 납니다. #Person1#는 브레이크를 교...,소음
497,dev_498,"#Person1#: 안녕하세요, 아마존 고객 서비스입니다. 무엇을 도와드릴까요?\n...",#Person2#님이 아마존 고객 서비스에 전화하여 아마존에서 받은 책에 한 페이지...,빠진 페이지
498,dev_499,#Person1#: 여름이 다 되어간다는 게 믿기지 않아.\r\n#Person2#:...,#Person2#는 #Person1#에게 여름 휴가 동안 파티를 도와주는 회사에서 ...,여름 휴가


## 1. 데이터 가공 및 데이터셋 클래스 구축
- csv file 을 불러와서 encoder 와 decoder의 입력형태로 가공해줍니다.
- 가공된 데이터를 torch dataset class 로 구축하여 모델에 입력가능한 형태로 만듭니다.

In [151]:
# 데이터 전처리를 위한 클래스: 데이터셋을 데이터프레임으로 변환하고 인코더와 디코더의 입력을 생성합니다.
class Preprocess:
    def __init__(self, prefix: str) -> None:
        self.prefix = prefix  # T5 모델에서 사용할 태스크 프리픽스 (예: "summarize: ")

    @staticmethod
    # 실험에 필요한 컬럼을 가져옵니다.
    def make_set_as_df(file_path, is_train=True):
        df = pd.read_csv(file_path)
        if is_train:
            train_df = df[['fname', 'dialogue', 'summary']]
            return train_df
        else:
            test_df = df[['fname', 'dialogue']]
            return test_df

    # T5 모델의 입력 형태에 맞게 전처리를 진행합니다.
    def make_input(self, dataset, is_test=False):
        if is_test:
            # 테스트 데이터셋의 경우, 입력 텍스트에 태스크 프리픽스를 추가합니다.
            encoder_input = dataset['dialogue'].apply(lambda x: self.prefix + str(x))
            decoder_input = [""] * len(dataset['dialogue'])  # T5 모델에서는 테스트 시 디코더 입력이 필요하지 않습니다.
            return encoder_input.tolist(), decoder_input
        else:
            # 학습 데이터셋의 경우, 입력 텍스트에 태스크 프리픽스를 추가하고, 출력 텍스트로 학습합니다.
            encoder_input = dataset['dialogue'].apply(lambda x: self.prefix + str(x))
            decoder_input = dataset['summary'].apply(lambda x: str(x))  # T5 모델에서는 디코더 입력을 별도로 준비하지 않습니다.
            decoder_output = dataset['summary'].apply(lambda x: str(x))  # 목표 요약 문장을 디코더 출력으로 사용합니다.
            return encoder_input.tolist(), decoder_input.tolist(), decoder_output.tolist()


In [152]:
from torch.utils.data import Dataset

# Train에 사용되는 Dataset 클래스를 정의합니다.
class DatasetForTrain(Dataset):
    def __init__(self, encoder_input, labels, length):
        self.encoder_input = encoder_input
        self.labels = labels
        self.length = length

    def __getitem__(self, idx):
        # 인코더 입력: input_ids와 attention_mask
        item = {key: val[idx].clone().detach() for key, val in self.encoder_input.items()}
        
        # 디코더의 labels는 요약문으로 설정
        item['labels'] = self.labels['input_ids'][idx].clone().detach()
        return item

    def __len__(self):
        return self.length

# Validation에 사용되는 Dataset 클래스를 정의합니다.
class DatasetForVal(Dataset):
    def __init__(self, encoder_input, labels, length):
        self.encoder_input = encoder_input
        self.labels = labels
        self.length = length

    def __getitem__(self, idx):
        # 인코더 입력: input_ids와 attention_mask
        item = {key: val[idx].clone().detach() for key, val in self.encoder_input.items()}
        
        # 디코더의 labels는 요약문으로 설정
        item['labels'] = self.labels['input_ids'][idx].clone().detach()
        return item

    def __len__(self):
        return self.length

# Test에 사용되는 Dataset 클래스를 정의합니다.
class DatasetForInference(Dataset):
    def __init__(self, encoder_input, test_id, length):
        self.encoder_input = encoder_input
        self.test_id = test_id
        self.length = length

    def __getitem__(self, idx):
        # 인코더 입력: input_ids와 attention_mask
        item = {key: val[idx].clone().detach() for key, val in self.encoder_input.items()}
        item['ID'] = self.test_id[idx]
        return item

    def __len__(self):
        return self.length


In [153]:
def prepare_train_dataset(config, preprocessor, data_path, tokenizer):
    train_file_path = os.path.join(data_path, 'train.csv')
    val_file_path = os.path.join(data_path, 'dev.csv')

    # Train, validation에 대해 각각 데이터프레임을 구축합니다.
    train_data = preprocessor.make_set_as_df(train_file_path)
    val_data = preprocessor.make_set_as_df(val_file_path)

    print('-' * 150)
    print(f'train_data:\n {train_data["dialogue"][0]}')
    print(f'train_label:\n {train_data["summary"][0]}')

    print('-' * 150)
    print(f'val_data:\n {val_data["dialogue"][0]}')
    print(f'val_label:\n {val_data["summary"][0]}')

    # 인코더 입력 및 출력 데이터를 준비합니다.
    encoder_input_train, _, decoder_output_train = preprocessor.make_input(train_data)
    encoder_input_val, _, decoder_output_val = preprocessor.make_input(val_data)
    print('-' * 10, 'Load data complete', '-' * 10)

    # 인코더 입력을 토크나이저로 토큰화합니다.
    tokenized_encoder_inputs = tokenizer(encoder_input_train, return_tensors="pt", padding=True,
                                         add_special_tokens=True, truncation=True, max_length=config['tokenizer']['encoder_max_len'], return_token_type_ids=False)

    # 디코더 출력을 토크나이저로 토큰화합니다 (T5 모델에서 labels로 사용).
    tokenized_labels = tokenizer(decoder_output_train, return_tensors="pt", padding=True,
                                 add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False)

    # 학습 데이터셋 생성
    train_inputs_dataset = DatasetForTrain(tokenized_encoder_inputs, tokenized_labels, len(encoder_input_train))

    # 검증 데이터의 인코더 입력을 토크나이저로 토큰화합니다.
    val_tokenized_encoder_inputs = tokenizer(encoder_input_val, return_tensors="pt", padding=True,
                                             add_special_tokens=True, truncation=True, max_length=config['tokenizer']['encoder_max_len'], return_token_type_ids=False)

    # 검증 데이터의 디코더 출력을 토크나이저로 토큰화합니다 (T5 모델에서 labels로 사용).
    val_tokenized_labels = tokenizer(decoder_output_val, return_tensors="pt", padding=True,
                                     add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False)

    # 검증 데이터셋 생성
    val_inputs_dataset = DatasetForVal(val_tokenized_encoder_inputs, val_tokenized_labels, len(encoder_input_val))

    print('-' * 10, 'Make dataset complete', '-' * 10)
    return train_inputs_dataset, val_inputs_dataset


## 2. Trainer 및 Trainingargs 구축하기
- Huggingface 의 Trainer 와 Training arguments를 활용하여 모델 학습을 일괄적으로 처리해주는 클래스를 정의합니다.

In [154]:
# 모델 성능에 대한 평가 지표를 정의합니다. 본 대회에서는 ROUGE 점수를 통해 모델의 성능을 평가합니다.
def compute_metrics(config,tokenizer,pred):
    rouge = Rouge()
    predictions = pred.predictions
    labels = pred.label_ids

    predictions[predictions == -100] = tokenizer.pad_token_id
    labels[labels == -100] = tokenizer.pad_token_id

    decoded_preds = tokenizer.batch_decode(predictions, clean_up_tokenization_spaces=True)
    labels = tokenizer.batch_decode(labels, clean_up_tokenization_spaces=True)

    # 정확한 평가를 위해 미리 정의된 불필요한 생성토큰들을 제거합니다.
    replaced_predictions = decoded_preds.copy()
    replaced_labels = labels.copy()
    remove_tokens = config['inference']['remove_tokens']
    for token in remove_tokens:
        replaced_predictions = [sentence.replace(token," ") for sentence in replaced_predictions]
        replaced_labels = [sentence.replace(token," ") for sentence in replaced_labels]

    print('-'*150)
    print(f"PRED: {replaced_predictions[0]}")
    print(f"GOLD: {replaced_labels[0]}")
    print('-'*150)
    print(f"PRED: {replaced_predictions[1]}")
    print(f"GOLD: {replaced_labels[1]}")
    print('-'*150)
    print(f"PRED: {replaced_predictions[2]}")
    print(f"GOLD: {replaced_labels[2]}")

    # 최종적인 ROUGE 점수를 계산합니다.
    results = rouge.get_scores(replaced_predictions, replaced_labels,avg=True)

    # ROUGE 점수 중 F-1 score를 통해 평가합니다.
    result = {key: value["f"] for key, value in results.items()}
    return result

In [155]:
# 학습을 위한 trainer 클래스와 매개변수를 정의합니다.
def load_trainer_for_train(config,generate_model,tokenizer,train_inputs_dataset,val_inputs_dataset):
    print('-'*10, 'Make training arguments', '-'*10,)
    # set training args
    training_args = Seq2SeqTrainingArguments(
                output_dir=config['general']['output_dir'], # model output directory
                overwrite_output_dir=config['training']['overwrite_output_dir'],
                num_train_epochs=config['training']['num_train_epochs'],  # total number of training epochs
                learning_rate=config['training']['learning_rate'], # learning_rate
                per_device_train_batch_size=config['training']['per_device_train_batch_size'], # batch size per device during training
                per_device_eval_batch_size=config['training']['per_device_eval_batch_size'],# batch size for evaluation
                warmup_ratio=config['training']['warmup_ratio'],  # number of warmup steps for learning rate scheduler
                weight_decay=config['training']['weight_decay'],  # strength of weight decay
                lr_scheduler_type=config['training']['lr_scheduler_type'],
                optim =config['training']['optim'],
                gradient_accumulation_steps=config['training']['gradient_accumulation_steps'],
                evaluation_strategy=config['training']['evaluation_strategy'], # evaluation strategy to adopt during training
                save_strategy =config['training']['save_strategy'],
                save_total_limit=config['training']['save_total_limit'], # number of total save model.
                fp16=config['training']['fp16'],
                load_best_model_at_end=config['training']['load_best_model_at_end'], # 최종적으로 가장 높은 점수 저장
                seed=config['training']['seed'],
                logging_dir=config['training']['logging_dir'], # directory for storing logs
                logging_strategy=config['training']['logging_strategy'],
                predict_with_generate=config['training']['predict_with_generate'], #To use BLEU or ROUGE score
                generation_max_length=config['training']['generation_max_length'],
                do_train=config['training']['do_train'],
                do_eval=config['training']['do_eval'],
                report_to=config['training']['report_to'] # (선택) wandb를 사용할 때 설정합니다.
            )

    # (선택) 모델의 학습 과정을 추적하는 wandb를 사용하기 위해 초기화 해줍니다.
    wandb.init(
        entity=config['wandb']['entity'],
        project=config['wandb']['project'],
        name=config['wandb']['name'],
    )

    # (선택) 모델 checkpoint를 wandb에 저장하도록 환경 변수를 설정합니다.
    os.environ["WANDB_LOG_MODEL"]="end"
    os.environ["WANDB_WATCH"]="false"

    # Validation loss가 더 이상 개선되지 않을 때 학습을 중단시키는 EarlyStopping 기능을 사용합니다.
    MyCallback = EarlyStoppingCallback(
        early_stopping_patience=config['training']['early_stopping_patience'],
        early_stopping_threshold=config['training']['early_stopping_threshold']
    )
    print('-'*10, 'Make training arguments complete', '-'*10,)
    print('-'*10, 'Make trainer', '-'*10,)

    # Trainer 클래스를 정의합니다.
    trainer = Seq2SeqTrainer(
        model=generate_model, # 사용자가 사전 학습하기 위해 사용할 모델을 입력합니다.
        args=training_args,
        train_dataset=train_inputs_dataset,
        eval_dataset=val_inputs_dataset,
        compute_metrics = lambda pred: compute_metrics(config,tokenizer, pred),
        callbacks = [MyCallback]
    )
    print('-'*10, 'Make trainer complete', '-'*10,)

    return trainer

In [156]:
from transformers import AutoTokenizer, AutoConfig, T5ForConditionalGeneration

# 학습을 위한 tokenizer와 사전 학습된 모델을 불러옵니다.
def load_tokenizer_and_model_for_train(config, device):
    print('-'*10, 'Load tokenizer & model', '-'*10,)
    print('-'*10, f'Model Name : {config["general"]["model_name"]}', '-'*10,)

    # 모델 이름 가져오기
    model_name = config['general']['model_name']

    # T5 모델 설정 로드
    t5_config = AutoConfig.from_pretrained(model_name)
    
    # 토크나이저 로드
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # T5 모델 로드
    generate_model = T5ForConditionalGeneration.from_pretrained(model_name, config=t5_config)

    # 스페셜 토큰 추가
    special_tokens_dict = {'additional_special_tokens': config['tokenizer']['special_tokens']}
    tokenizer.add_special_tokens(special_tokens_dict)

    # 추가된 토큰 수에 맞춰 모델의 토큰 임베딩 크기 재조정
    generate_model.resize_token_embeddings(len(tokenizer))

    # 모델을 지정된 장치로 이동
    generate_model.to(device)
    print(generate_model.config)

    print('-'*10, 'Load tokenizer & model complete', '-'*10,)
    return generate_model, tokenizer


## 3. 모델 학습하기

- 앞에서 구축한 클래스 및 함수를 활용하여 학습 진행합니다.

In [157]:
def main(config):
    # 사용할 device를 정의합니다.
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print('-' * 10, f'device : {device}', '-' * 10)
    print(torch.__version__)

    # 사용할 모델과 tokenizer를 불러옵니다.
    generate_model, tokenizer = load_tokenizer_and_model_for_train(config, device)
    print('-' * 10, "tokenizer special tokens : ", tokenizer.special_tokens_map, '-' * 10)

    # 학습에 사용할 데이터셋을 불러옵니다.
    # T5 모델은 bos_token과 eos_token을 사용하지 않으므로, prefix를 전달합니다.
    preprocessor = Preprocess(prefix=config['tokenizer'].get('prefix', 'summarize: ')) 
    data_path = config['general']['data_path']
    train_inputs_dataset, val_inputs_dataset = prepare_train_dataset(config, preprocessor, data_path, tokenizer)

    # Trainer 클래스를 불러옵니다.
    trainer = load_trainer_for_train(config, generate_model, tokenizer, train_inputs_dataset, val_inputs_dataset)
    trainer.train()   # 모델 학습을 시작합니다.

    # (선택) 모델 학습이 완료된 후 wandb를 종료합니다.
    wandb.finish()


In [158]:
if __name__ == "__main__":
    main(loaded_config)

---------- device : cuda:0 ----------
2.4.0+cu121
---------- Load tokenizer & model ----------
---------- Model Name : eenzeenee/t5-base-korean-summarization ----------


/opt/conda/envs/cv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


T5Config {
  "_name_or_path": "eenzeenee/t5-base-korean-summarization",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "max_length": 128,
  "model_type": "t5",
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "torch_dtype": "float32",
  "transformers_version": "4.44.2",
  "use_cache": true,
  "vocab_size": 50374
}

---------- Load tokenizer & model complete ----------
---------- tokenizer special tokens :  {'eos_token': '</s>', 'unk_token': '<pad>', 'pad_token': '<pad>',

/opt/conda/envs/cv/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


/opt/conda/envs/cv/lib/python3.10/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


---------- Make training arguments complete ----------
---------- Make trainer ----------
---------- Make trainer complete ----------


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

: 

## 4. 모델 추론하기

In [ ]:
# 이곳에 내가 사용할 wandb config 설정
loaded_config['inference']['ckt_path'] = "/root/output/checkpoint-3116/"

- test data를 사용하여 모델의 성능을 확인합니다.

In [ ]:
# tokenization 과정까지 진행된 최종적으로 모델에 입력될 데이터를 출력합니다.
def prepare_test_dataset(config, preprocessor, tokenizer):
    # 테스트 데이터 경로 설정
    test_file_path = os.path.join(config['general']['data_path'], 'test.csv')

    # 테스트 데이터셋 준비
    test_data = preprocessor.make_set_as_df(test_file_path, is_train=False)
    test_id = test_data['fname']

    print('-' * 150)
    print(f'test_data:\n{test_data["dialogue"][0]}')
    print('-' * 150)

    # 인코더 입력 준비 (T5 모델에서는 디코더 입력을 따로 준비하지 않음)
    encoder_input_test, _ = preprocessor.make_input(test_data, is_test=True)
    print('-' * 10, 'Load data complete', '-' * 10,)

    # 인코더 입력을 토크나이저로 토큰화합니다.
    test_tokenized_encoder_inputs = tokenizer(
        encoder_input_test,
        return_tensors="pt",
        padding=True,
        add_special_tokens=True,
        truncation=True,
        max_length=config['tokenizer']['encoder_max_len'],
        return_token_type_ids=False,
    )

    # 테스트 데이터셋 생성
    test_encoder_inputs_dataset = DatasetForInference(test_tokenized_encoder_inputs, test_id, len(encoder_input_test))
    print('-' * 10, 'Make dataset complete', '-' * 10,)

    return test_data, test_encoder_inputs_dataset

In [ ]:
from transformers import AutoTokenizer, T5ForConditionalGeneration

# 추론을 위한 tokenizer와 학습시킨 모델을 불러옵니다.
def load_tokenizer_and_model_for_test(config, device):
    print('-'*10, 'Load tokenizer & model', '-'*10)

    # 모델과 토크나이저 저장된 경로 설정
    tokenizer_path = 'root/saved_tokenizer'
    model_path = 'root/saved_model'

    # 저장된 토크나이저 로드
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

    # 저장된 모델 로드
    generate_model = T5ForConditionalGeneration.from_pretrained(model_path, ignore_mismatched_sizes=True)

    # 스페셜 토큰이 이미 추가되어 있으므로 다시 추가할 필요 없음

    # 모델을 지정한 장치로 이동
    generate_model.to(device)
    print('-'*10, 'Load tokenizer & model complete', '-'*10)

    return generate_model, tokenizer


In [ ]:
import os
import torch
import pandas as pd
from tqdm import tqdm
from torch.utils.data import DataLoader

# 학습된 모델이 생성한 요약문의 출력 결과를 보여줍니다.
def inference(config):
    # 사용할 장치를 정의합니다.
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print('-'*10, f'device : {device}', '-'*10)
    print(torch.__version__)

    # 모델과 토크나이저를 로드합니다.
    generate_model, tokenizer = load_tokenizer_and_model_for_test(config, device)

    # 데이터 경로와 전처리 객체를 정의합니다.
    data_path = config['general']['data_path']
    preprocessor = Preprocess(prefix=config['tokenizer'].get('prefix', 'summarize: '))

    # 테스트 데이터셋을 준비합니다.
    test_data, test_encoder_inputs_dataset = prepare_test_dataset(config, preprocessor, tokenizer)
    dataloader = DataLoader(test_encoder_inputs_dataset, batch_size=config['inference']['batch_size'])

    summary = []
    text_ids = []

    # 추론을 수행합니다.
    with torch.no_grad():
        for item in tqdm(dataloader):
            text_ids.extend(item['ID'])
            input_ids = item['input_ids'].to(device)
            
            generated_ids = generate_model.generate(
                input_ids=input_ids,
                no_repeat_ngram_size=config['inference']['no_repeat_ngram_size'],
                early_stopping=config['inference']['early_stopping'],
                max_length=config['inference']['generate_max_length'],
                num_beams=config['inference']['num_beams'],
            )
            
            # 요약문 디코딩 및 저장
            for ids in generated_ids:
                result = tokenizer.decode(ids, skip_special_tokens=False, clean_up_tokenization_spaces=True)

                # 불필요한 스페셜 토큰 제거
                result = result.replace('<pad>', '').replace('</s>', '').strip()
                summary.append(result)

    # 결과를 데이터프레임으로 변환하고 파일로 저장
    output = pd.DataFrame({"fname": test_data['fname'], "summary": summary})
    result_path = config['inference']['result_path']
    
    if not os.path.exists(result_path):
        os.makedirs(result_path)
    
    output.to_csv(os.path.join(result_path, "output_t5_remove_pad_more_specialtoken_epoch100.csv"), index=False)

    return output


In [ ]:
# 학습된 모델의 test를 진행합니다.
if __name__ == "__main__":
    output = inference(loaded_config)

---------- device : cuda:0 ----------
2.4.0+cu121
---------- Load tokenizer & model ----------
---------- Load tokenizer & model complete ----------
------------------------------------------------------------------------------------------------------------------------------------------------------
test_data:
#Person1#: 더슨 씨, 받아쓰기 좀 해주세요. 
#Person2#: 네, 실장님...
#Person1#: 이것은 오늘 오후까지 모든 직원에게 내부 메모로 전달되어야 합니다. 준비되셨나요?
#Person2#: 네, 실장님. 시작하셔도 됩니다.
#Person1#: 모든 직원들에게 주의하라... 즉시 효력을 발휘하여, 모든 사무실 통신은 이메일 통신과 공식 메모로 제한됩니다. 근무 시간 동안 직원들이 즉시 메시지 프로그램을 사용하는 것은 엄격히 금지됩니다.
#Person2#: 실장님, 이것은 내부 통신에만 적용되는 건가요? 아니면 외부 통신에도 제한이 되는 건가요?
#Person1#: 이것은 모든 통신에 적용되어야 합니다, 이 사무실 내의 직원들 사이뿐만 아니라 외부 통신에도 마찬가지입니다.
#Person2#: 하지만 실장님, 많은 직원들이 고객과 소통하기 위해 즉시 메시지를 사용하고 있습니다.
#Person1#: 그들은 그들의 의사소통 방법을 바꾸어야만 합니다. 이 사무실에서 누구도 즉시 메시지를 사용하지 않기를 원합니다. 너무 많은 시간을 낭비하게 됩니다! 이제, 메모를 계속해주세요. 우리가 어디까지 했나요?
#Person2#: 이것은 내부와 외부 통신에 적용됩니다.
#Person1#: 그렇습니다. 즉시 메시지를 계속 사용하는 어떤 직원이라도 먼저 경고를 받고 직무 정지에 처해질 것입니다. 두 번째 위반 시에

100%|██████████| 16/16 [00:56<00:00,  3.55s/it]


In [ ]:
output  # 각 대화문에 대한 요약문이 출력됨을 확인할 수 있습니다.

,fname,summary
0,test_0,"은 에게 모든 직원에게의 내부 메모를 받아쓰기 하도록 요청하고, 즉시 효력을 발휘하..."
1,test_1,"는 교통 체증에 걸렸다. 은 에게 대중교통을 이용하라고 제안하고, 날씨가 좋을 때 ..."
2,test_2,케이트는 마샤와 히어로가 2개월 동안 별거 중이다가 이혼을 신청했다고 에게 말한다....
3,test_3,브라이언은 의 생일을 축하하기 위해 파티를 즐기고 있다. 는 브라이언에게 함께 술 ...
4,test_4,"는 에게 올림픽 스타디움이 6월에 완공될 예정이며, 총 5000개의 좌석을 가지고 ..."
...,...,...
494,test_495,"찰리는 잭에게 학교 끝나고 집에 와서 비디오 게임을 하자고 제안하고, 그는 그것이 ..."
495,test_496,은 에게 컨트리 음악에 관심을 가지게 된 계기와 라디오 방송국에서 일하기 시작한 이...
496,test_497,앨리스는 기계를 어떻게 사용하는지 에게 알려줍니다. 은 앨리스가 옷을 깨끗하게 하고...
497,test_498,"스티브는 매튜에게 최근에 살 곳을 찾고 있으며, 그의 이웃인 다우 부인이 그녀와 함..."
